### Importing necessary libs

In [1]:
from sentence_transformers import SentenceTransformer
from Utility.ThesisUtility import mbkmeans_clusters,writeClustersToExcel,getSentenceUsingStructringWords,printPairsEvaluation,printPurity,getSentcesAndTokens

###  Constants

In [2]:
dataFilePath = "../Dataset/Essays_all/TelevisonMergedT1+T2"
evaluationFilePath = "../CSV Files/Clusters Evaluation results/GOLD vs Clusters - SBERT.xlsx"
k = 26 # the value for the K-Means
modelFileName = "SbertSentences.txt"
clusterExcelSheetName = "clusteredArgument- Sbert - "
#defining the model
model = SentenceTransformer('all-mpnet-base-v2', device="cuda")

### Reading the essays from the directory

In [3]:
sentences = getSentcesAndTokens (dataFilePath,False)

### Transorming the text of the sentences into vectors

In [4]:
corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")

Batches:   0%|          | 0/1210 [00:00<?, ?it/s]

### applying the clustering method

In [ ]:
clustering, cluster_labels = mbkmeans_clusters(X=corpus_embeddings, k=k, print_silhouette_values=True)

writeClustersToExcel(clusterExcelSheetName,cluster_labels,k,sentences,corpus_embeddings,clustering)

### Evaluating of the clusters

In [ ]:
#Pairs evaluation metric
printPairsEvaluation(evaluationFilePath,"excel")

#Purity evaluation metric
printPurity(evaluationFilePath,"excel")